# Joint Bert Model for slot and intent classification

### Imports

In [172]:
import json
import os
import re
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, CategoricalAccuracy

model_name = "bert-base-uncased"

### Load Dataset

In [173]:
inputs = []
intentOutputs = []
slotOutputs = []

with open("../processing/JERTmate_final_data.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

    inputs = data["inputs"]
    intentOutputs = data["intentOutputs"]
    slotOutputs = data["slotOutputs"]

### Split Data - Train 80% | Validation 10% | Test 10%

In [174]:
def split_arrays(inputs, intentOutputs, slotOutputs, train_ratio, val_ratio, test_ratio, seed=42):
    assert len(inputs) == len(intentOutputs) == len(slotOutputs), "All arrays must have the same length"
    
    np.random.seed(seed)
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    
    n_total = len(inputs)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    
    inputs_train, inputs_val, inputs_test = inputs[:n_train], inputs[n_train:n_train + n_val], inputs[n_train + n_val:]
    intentOutputs_train, intentOutputs_val, intentOutputs_test = intentOutputs[:n_train], intentOutputs[n_train:n_train + n_val], intentOutputs[n_train + n_val:]
    slotOutputs_train, slotOutputs_val, slotOutputs_test = slotOutputs[:n_train], slotOutputs[n_train:n_train + n_val], slotOutputs[n_train + n_val:]
    
    return (tf.constant(inputs_train), tf.constant(inputs_val), tf.constant(inputs_test)), (tf.constant(intentOutputs_train), tf.constant(intentOutputs_val), tf.constant(intentOutputs_test)), (tf.constant(slotOutputs_train), tf.constant(slotOutputs_val), tf.constant(slotOutputs_test))

train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

(inputs_train, inputs_val, inputs_test), (intentOutputs_train, intentOutputs_val, intentOutputs_test), (slotOutputs_train, slotOutputs_val, slotOutputs_test) = split_arrays(inputs, intentOutputs, slotOutputs, train_ratio, val_ratio, test_ratio)

print(slotOutputs_train.shape)
print(intentOutputs_train.shape)

(10324, 330)
(10324, 38)


### Define Model

In [175]:
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_vector_length=None, slot_vector_length=None, model_name=model_name, dropout_prob=0.08):
        super().__init__(name="joint_intent_slot")
        #   ** GENERAL LAYERS **
        self.bert = TFBertModel.from_pretrained(model_name) # BERT model
        self.dropout = Dropout(dropout_prob) # basic dropout layer



        #   ** SLOT LAYERS **
        # LHS compressor
        self.conv_layer = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 768), strides=(1, 1),padding='valid',activation='relu')
        #self.sequence_output_processor = Dense(150, activation="relu", name="sequence_output_processor")

        # processing layers
        self.slot_processor = Dense(((150) + (intent_vector_length * 2) + (slot_vector_length * 2)), activation="relu", name="slot_processor")

        # output layer
        self.slot_output = Dense(slot_vector_length, activation='relu', name="slot_output")



        #  ** INTENT LAYERS **
        # processing layers
        self.intent_processor = Dense(((150 * 1) + (intent_vector_length * 2)), activation="relu", name="intent_processor")

        # output layer
        self.intent_output = Dense(intent_vector_length, activation='softmax', name="intent_output")

    def __call__(self, inputs, **kwargs):
        # run BERT
        trained_bert = self.bert(inputs[:, :150], **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state

        #   ** SLOT CLASSIFICATION **
        # reduce dimensionality of sequence output - Mini CNN - [batch_size, sequence_length, 1]
        conv_output = self.conv_layer(tf.expand_dims(sequence_output, axis=-1))

        # flatten sequence output
        compressed_sequence_output = tf.squeeze(conv_output, axis=[-1, -2])

        # slot processor
        slot_processor_input = self.dropout(tf.concat([compressed_sequence_output, tf.cast(inputs[:, 150:], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        slot_processor_output = self.slot_processor(slot_processor_input)

        # slot output
        slot_output_input = self.dropout(slot_processor_output, training=kwargs.get("training", False))
        slot_logits = self.slot_output(slot_output_input)
        print(slot_logits.shape)



        #   ** INTENT CLASSIFICATION **
        # intent processor
        intent_processor_input = self.dropout(tf.concat([pooled_output, tf.cast(inputs[:, 150:150 + 114], dtype=tf.float32)], axis=-1), training=kwargs.get("training", False))
        intent_processor_output = self.intent_processor(intent_processor_input)

        # intent output
        intent_output_input = self.dropout(intent_processor_output, training=kwargs.get("training", False))
        intent_logits = self.intent_output(intent_output_input)
        print(intent_logits.shape)

        return slot_logits, intent_logits

    def get_config(self):
        config = super(JointIntentAndSlotFillingModel, self).get_config()
        config.update({
            "dropout": self.dropout_prob,
            "intent_vector_length": self.intent_vector_length,
            "slot_vector_length": self.slot_vector_length,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
joint_model = JointIntentAndSlotFillingModel(intent_vector_length=38, slot_vector_length=330)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

### Train and Compile

In [176]:
# optimizer
opt = Adam(learning_rate=5e-5, epsilon=1e-08)

# learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 7:
        return lr
    else:
        return lr * 0.97
learning_rate_callback = LearningRateScheduler(scheduler)

# Model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath="model_epoch_{epoch:02d}.keras",  # Save the model with the epoch number in the filename
    save_freq='epoch',
    verbose=1 
)

# two losses, one for slots, another for intents
losses = [SparseCategoricalCrossentropy(from_logits=True, name="slot_loss"), CategoricalCrossentropy(from_logits=True, name="intent_loss")]
metrics = [SparseCategoricalAccuracy(name="slot_accuracy"), CategoricalAccuracy(name="intent_accuracy")]

# compile model
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

history = joint_model.fit(
    x=inputs_train, 
    y=(slotOutputs_train, intentOutputs_train), 
    validation_data=(inputs_val, (slotOutputs_val, intentOutputs_val)), 
    epochs=10, 
    batch_size=16,
    shuffle=True, 
    callbacks=[learning_rate_callback, checkpoint_callback],
)

Epoch 1/10
(None, 330)
(None, 38)


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 330), output.shape=(None, 330)

### Evaluate Model

keep in mind -> test slot accuracy will be higher than reality.
        Because 90% of the data points are 0, it can just guess 0 and be right 85% of the time

In [ ]:
test_loss, test_slot_acc, test_intent_acc = joint_model.evaluate(x=inputs_test, y=(slotOutputs_test, intentOutputs_test), batch_size=32)

print(f"Test Slot Accuracy: {test_slot_acc}")
print(f"Test Intent Accuracy: {test_intent_acc}")

### Predict

In [20]:
def nlu(text, tokenizer, model, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = model(inputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]

    info = {"intent": intent_names[intent_id], "slots": {}}

    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([re.sub(r'^[BI]-', '', slot_names[s]) for s in slot_ids if s != 0])
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    if text[0].islower():
      tokens = tokenizer.tokenize(text, add_special_tokens=True)
    else:
      tokens = tokenizer.tokenize(text)

    # process sequence output for slots
    for token, slot_id in zip(tokens, slot_ids):
      # add all to out_dict
      slot_name = slot_names[slot_id]

      if slot_name == "[PAD]":
        continue

      # collect tokens
      collected_tokens = [token]
      idx = tokens.index(token)

      # see if it starts with ##
      # then it belongs to the previous token
      if token.startswith("##"):

        # check if the token already exists or not
        if tokens[idx - 1] not in out_dict[slot_name]:
          collected_tokens.insert(0, tokens[idx - 1])

      # add collected tokens to slots
      out_dict[slot_name].extend(collected_tokens)

    # process out_dict
    for slot_name in out_dict:
      tokens = out_dict[slot_name]
      slot_value = tokenizer.convert_tokens_to_string(tokens)

      info["slots"][slot_name] = slot_value.strip()

    return info

In [ ]:
#loaded_model = tf.keras.models.load_model('model_epoch_18.keras', custom_objects={'JointIntentAndSlotFillingModel': JointIntentAndSlotFillingModel})

print(nlu("could you please change my reservation on december twenty first originally scheduled for twelve thirty in the morning to now be at four thirty five pm", tokenizer, joint_model, intent_names, slot_names))